### Important imports

In [1]:
from scipy import stats
import os
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np  # fundamental package for acientific computing with python
import matplotlib 
from matplotlib import pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
%matplotlib inline

from plotly import tools

init_notebook_mode(connected=True)

In [4]:
# The function to plot the distribution of the categorical values Horizontaly 
def bar_hor(df,  col, title, color, w=None, h=None, lm=0,  limit=100, return_trace=False, rev=False, xlb=False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1]
    xx = cnt_srs.head(limit).values[::-1]
    if rev:
        yy = cnt_srs.tail(limit).index[::-1]
        xx = cnt_srs.tail(limit).values[::-1]
    if xlb:#????
        trace = go.Bar(y=xlb, x=xx,orientation='h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx,orientation='h', marker=dict(color=color))
    if return_trace:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [5]:
# The function to get the distribution of the categories according to the target
#(target de dtype=bool? or np.int8?)
def gp(df, col, title):
    df0 = df[df['label']==0]
    df1 = df[df['label']==1]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(df[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100/total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100/total[x1[i]] for i,x in enumerate(b1.values)]
    
    trace1 = go.Bar(x=x0, y=y0, name="Target : 0", marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=x1, y=y1, name="Target : 1", marker=dict(color="#FEBFB3"))
    
    return trace1, trace2

In [6]:
def exploreCat(df, col):
    t = df[col].value_counts()
    labels = t.index 
    values = t.values
    colors = ["#96D38C",  "#FEBFB3"]
    trace  = go.Pie(labels=labels, values=values,
                   hoverinfo="all",textinfo='value',
                   textfont=dict(size=12), 
                   marker=dict(colors=colors,
                               line=dict(color='#fff',width=2)))
    layout = go.Layout(title=col, height=400)
    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)

# the relation between the categorical column and the target
def catAndTrgt(df, col):
    tr0 = bar_hor(df, col, "Distribution of "+col, "#f975ae", w=700, lm=100, return_trace=True)
    tr1, tr2 = gp(df, col, "Distribution of Target with "+col)
    
    fig = tools.make_subplots(rows=1, cols=3, print_grid=False, 
                             subplot_titles=[col+" Distribution", "% of target=0", "% of target=1"])
    fig.append_trace(tr0, 1, 1);
    fig.append_trace(tr1, 1, 2);
    fig.append_trace(tr2, 1, 3);
    fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
    iplot(fig);

In [7]:
merge_df = pd.read_excel("data/路虎口碑所有用户.xlsx")
# print(merge_df.describe())
merge_df.head(2)

,user_id,follow_count,fans_count,gender,birthday,location,level,car_liked,registration_time,properties,mileage,post,cars,koubei_post,gas_mileage,car_friend_zone,label,excellent_post_count,all_post_count,car_like
0,yryihgddsg,0,1,man,NaT,浙江 杭州,1,关注的车：驭胜S330,2017-11-08,普通用户,1870,NaN,NaN,"[{'title': '2018款极光骚红色', 'url': '//k.autohome....",NaN,NaN,1,0,0,驭胜S330
1,淡淡可,2,1,man,NaT,山东 济宁,1,关注的车：博越,2017-11-10,普通用户,590,NaN,NaN,"[{'title': '比邻全国最低价提车', 'url': '//k.autohome.c...",NaN,NaN,1,0,0,博越


#### Check missing data

In [8]:
print(merge_df.shape)
total = merge_df.isnull().sum().sort_values(ascending=False)
percentage = (merge_df.isnull().sum()/merge_df.isnull().count()*100).sort_values(ascending=False)
missing_use_luhu_data = pd.concat([total,percentage], axis=1,keys=['total', 'missing_percentage'])
missing_use_luhu_data.head(18)

(2569, 20)


,total,missing_percentage
gas_mileage,2389,92.993383
car_friend_zone,2195,85.441806
birthday,2162,84.157260
post,1722,67.029973
koubei_post,1682,65.472947
cars,1657,64.499805
car_like,70,2.724796
level,0,0.000000
follow_count,0,0.000000
fans_count,0,0.000000


In [9]:
dtypes = {'user_id':str, 
          'follow_count': int, 
          'fans_count':int,
          'gender':bool,
          'location':int,
          'level':int,
          'post_count':int,
          'car_liked':str,
          'registration_time':int,
          'properties':bool,
          'mileage':int,
          'post':str,
          'cars':str,
          'koubei_post':str,
          'gas_mileage':int,
          'car_friend_zone':str,
          'label': bool,
         }

In [10]:
merge_df.dtypes

user_id                         object
follow_count                     int64
fans_count                       int64
gender                          object
birthday                datetime64[ns]
location                        object
level                            int64
car_liked                       object
registration_time       datetime64[ns]
properties                      object
mileage                          int64
post                            object
cars                            object
koubei_post                     object
gas_mileage                     object
car_friend_zone                 object
label                            int64
excellent_post_count             int64
all_post_count                   int64
car_like                        object
dtype: object

### Distribution of label

In [11]:
# temp = merge_df['label'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df)
# df.plot(kind='pie',x='labels',y='values', title="buy luhu or not")
bar_hor(merge_df, "label", "Distribution of label", color=['#44ff54','#ff4444'], h=350, w=600,
       lm=200, xlb=['label : 0', "label : 1"])

In [12]:
# temp = merge_df['label'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df)
# plt.figure(figsize=(5,5))
# labels = temp.index
# plt.pie(x=temp.values,labels=labels,autopct='%1.1f%%',shadow=True)
# plt.show()

### Distribution of gender

In [13]:
# temp = merge_df['gender'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df)
# plt.figure(figsize=(5,5))
# labels = temp.index
# plt.pie(x=temp.values,labels=labels,autopct='%1.1f%%',shadow=True)
# plt.show()

In [14]:
exploreCat(merge_df, "gender")

### Distibution of follow_count, fans_count

In [15]:
# temp = merge_df['follow_count'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df.loc[:15])
# plt.figure(figsize=(10,5))
# labels = temp.index
# plt.bar(temp.index,temp.values)
# plt.show()

In [16]:
# temp = merge_df['fans_count'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df.loc[:10])
# plt.figure(figsize=(10,5))
# labels = temp.index
# plt.bar(temp.index,temp.values)
# plt.show()

In [17]:
catAndTrgt(merge_df, "follow_count")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




### Distibution of location

In [18]:
# temp = merge_df['location'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# df = df.sort_values('values', ascending=False)
# print(df.loc[0:10])
# ax = df['values'][0:10].plot(kind='bar')
# # plt.figure(figsize=(12,5))
# # plt.bar(df['labels'][:20],df['values'][:20])
# # plt.show()

In [19]:
catAndTrgt(merge_df,  "location")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




### Distibution of level

In [20]:
# temp = merge_df['level'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# print(df)
# plt.figure(figsize=(5,5))
# labels = temp.index
# plt.bar(temp.index,temp.values)
# plt.show()

In [21]:
catAndTrgt(merge_df, "level")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




### Mileage

In [22]:
# temp = merge_df['mileage'].value_counts()
# df = pd.DataFrame({"labels":temp.index,
#                    "values":temp.values
#                   })
# # print(df)
# df = df.sort_values('values', ascending=False)
# print(df.loc[0:10])
# ax = df['values'][0:10].plot(kind='bar')

In [23]:
catAndTrgt(merge_df, "mileage")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




### all_post_count 

In [24]:
catAndTrgt(merge_df, "all_post_count")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




In [25]:
catAndTrgt(merge_df, "excellent_post_count")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




In [26]:
catAndTrgt(merge_df, "car_like")

/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:318: DeprecationWarning:

plotly.graph_objs.Font is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Font
  - plotly.graph_objs.layout.hoverlabel.Font
  - etc.




In [27]:
merge_df.dtypes

user_id                         object
follow_count                     int64
fans_count                       int64
gender                          object
birthday                datetime64[ns]
location                        object
level                            int64
car_liked                       object
registration_time       datetime64[ns]
properties                      object
mileage                          int64
post                            object
cars                            object
koubei_post                     object
gas_mileage                     object
car_friend_zone                 object
label                            int64
excellent_post_count             int64
all_post_count                   int64
car_like                        object
dtype: object

In [ ]:
# cars
cars = merge_df

### feature extaction

In [ ]:
# mapping string to categorical number